In [1]:
import os
import sys

In [2]:
# Py4JJavaError 해결
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
from pyspark import SparkConf, SparkContext

In [4]:
conf = SparkConf().setMaster("local").setAppName("transformations_actions")
sc = SparkContext(conf = conf)

In [5]:
# 설정확인
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.app.startTime', '1677817480322'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'transformations_actions'),
 ('spark.driver.host', 'LAPTOP-O667B512'),
 ('spark.app.id', 'local-167781748558

In [6]:
# 중단
sc.stop()

In [7]:
type(sc)

pyspark.context.SparkContext

In [8]:
sc

AttributeError: 'NoneType' object has no attribute 'sc'

<SparkContext master=local appName=transformations_actions>

In [9]:
sc = SparkContext(conf=conf)

In [10]:
sc

<SparkContext master=local appName=transformations_actions>

In [11]:
# RDD 만들기
foods = sc.parallelize(["짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", 
                        "짬뽕", "짜장면", "짜장면", "짜장면",  "라면", "우동", "라면"])

In [12]:
foods

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [13]:
# RDD에 어떤 데이터가 있는지 확인
# 데이터를 모두 가져와서 사용하는데 실제 환경에서는 지양해야함
foods.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [14]:
foods.countByValue()

defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '짬뽕': 2,
             '떡볶이': 1,
             '쌀국수': 1,
             '라면': 2,
             '우동': 1})

In [15]:
# N개 데이터 뽑는다
foods.take(3)

['짜장면', '마라탕', '짬뽕']

In [16]:
# 첫번째 데이터 뽑는다
foods.first()

'짜장면'

In [17]:
# 전체 개수
foods.count()

12

In [18]:
# 유니크한 값 에 개수
foods.distinct().count()

7

In [22]:
foods.distinct().collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '라면', '우동']

In [19]:
# foreach : 요소를 하나씩 꺼내서 함수 적용
# print 안된다 > action이라 work load에 실행되어 여기 local에서는 보이지 않는다
# 로그 남길때 유용
foods.foreach(lambda x: print(x))

In [23]:
sc.parallelize([1, 2, 3]).map(lambda x: x + 2).collect()

[3, 4, 5]

In [24]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [25]:
moviesRDD = sc.parallelize(movies)

In [26]:
moviesRDD.collect()

['그린 북',
 '매트릭스',
 '토이 스토리',
 '캐스트 어웨이',
 '포드 V 페라리',
 '보헤미안 랩소디',
 '빽 투 더 퓨처',
 '반지의 제왕',
 '죽은 시인의 사회']

In [27]:
# flatMap : 리스트 요소 펼쳐볼때(flat)
# 공백을 기준으로 나누기
flatMovies = moviesRDD.flatMap(lambda x: x.split(" "))

In [28]:
flatMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [29]:
filteredMovies = flatMovies.filter(lambda x: x != "매트릭스")

In [30]:
filteredMovies.collect()

['그린',
 '북',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [31]:
num1 = sc.parallelize([1, 2, 3, 4])
num2 = sc.parallelize([4, 5, 6, 7, 8, 9, 10])

In [32]:
# 교집합
num1.intersection(num2).collect()

[4]

In [33]:
# 합집합 중복 상관없다
num1.union(num2).collect()

[1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10]

In [34]:
# 차집합
num1.subtract(num2).collect()

[2, 1, 3]

In [35]:
numUnion = num1.union(num2)

In [36]:
numUnion.collect()

[1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10]

In [37]:
# numUnion.sample(T)
# [1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10]
# sample : 무작위로 값 가져오기
# True/False : 내가 뽑은 값을 넣고 추첨할거냐 빼고 추첨하느냐
# True : 빼고 추첨
# 몇 퍼센트 추출? 0.5 >  50%
numUnion.sample(True, .5).collect()

[2, 2, 3, 4, 5, 5, 6, 6, 10, 10]

In [39]:
# 무작위라 항상 값, 값 길이가 다르다 > 고정하고 싶다면 seed 사용하기
numUnion.sample(True, .5, seed=5).collect()

[1, 4, 6, 9, 9, 10]

In [40]:
foods = sc.parallelize(["짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면",  "라면", "우동", "라면", "치킨", "돈까스", "회", "햄버거", "피자"])

In [41]:
# 첫 글자 기준으로 나누기
foodsGroup = foods.groupBy(lambda x: x[0])
res = foodsGroup.collect()
res

[('짜', <pyspark.resultiterable.ResultIterable at 0x1a8d52af860>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x1a8d52afe10>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x1a8d52af2e8>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x1a8d52afc50>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x1a8d52af128>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x1a8d52afda0>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x1a8d52afcc0>),
 ('치', <pyspark.resultiterable.ResultIterable at 0x1a8d52af940>),
 ('돈', <pyspark.resultiterable.ResultIterable at 0x1a8d528ee48>),
 ('회', <pyspark.resultiterable.ResultIterable at 0x1a8d528ecc0>),
 ('햄', <pyspark.resultiterable.ResultIterable at 0x1a8d528e400>),
 ('피', <pyspark.resultiterable.ResultIterable at 0x1a8d528e208>)]

In [42]:
for (k, v) in res:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
마 ['마라탕']
짬 ['짬뽕', '짬뽕']
떡 ['떡볶이']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']
치 ['치킨']
돈 ['돈까스']
회 ['회']
햄 ['햄버거']
피 ['피자']


In [43]:
nums = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

In [45]:
list(nums.groupBy(lambda x: x % 2).collect())

[(1, <pyspark.resultiterable.ResultIterable at 0x1a8d52aacc0>),
 (0, <pyspark.resultiterable.ResultIterable at 0x1a8d52aa898>)]

In [44]:
list(nums.groupBy(lambda x: x % 2).collect()[1][1])

[2, 4, 6, 8, 10]